In [ ]:
!pip install snowflake-opendic==0.1.11

## Setting up pyspark-opendic

In [2]:
from snowflake_opendic.snow_opendic import snowflake_connect

def read_secret(secret_name):
    """ Get `secret_name` from docker-compose secret store"""
    secret_path = f"/run/secrets/{secret_name}"
    try:
        with open(secret_path, "r") as f:
            return f.read().strip()  # Remove any trailing newline
    except FileNotFoundError:
        print(f"Secret {secret_name} not found.")
        return None

ENGINEER_CLIENT_ID = read_secret("engineer_client_id")
ENGINEER_CLIENT_SECRET = read_secret("engineer_client_secret")

print("Secrets read ✔️")

config_path = f"/run/secrets/snowflake-conf"
SNOWFLAKE_CONN = snowflake_connect(config_path)

print("Snowflake conn initialized ✔️")

Secrets read ✔️
Snowflake conn initialized ✔️


In [3]:
from snowflake_opendic.catalog import OpenDicSnowflakeCatalog


POLARIS_URI= "http://polaris:8181/api"

catalog = OpenDicSnowflakeCatalog(SNOWFLAKE_CONN, POLARIS_URI, ENGINEER_CLIENT_ID, ENGINEER_CLIENT_SECRET)
print("Catalog initialized ✔️")


Connection Established | Server: America/Los_Angeles | Latency: 0.346566 ✔︎
Catalog initialized ✔️


In [ ]:
catalog.sql("Show databases")

## Define the schema for a andfunc object

In [4]:
catalog.sql(
    """
    DEFINE OPEN function_v2
    props {
        "args": "MAP",
        "language": "STRING",
        "def": "string",
        "comment": "string",
        "packages": "list",
        "runtime": "string",
        "client_version": "int",
        "signature": "STRING",
        "return_type": "STRING"
    }
    """
)

```json
{
    "error": "HTTP Error",
    "details": "409 Client Error: Conflict for url: http://polaris:8181/api/opendic/v1/objects",
    "Catalog Response": null
}
```

In [5]:
catalog.sql(
 """
 CREATE OPEN function_v2 baz
    props {
            "args": {
                "arg1": "int", 
                "arg2": "int"
                },
            "language": "python",
            "def": "def foo(arg1, arg2):\\n\\n        return arg1 + arg2",
            "packages" : ["numpy", "pandas"],
            "comment": "test fun",
            "runtime": "3.12",
            "client_version": 1,
            "return_type": "int",
            "signature": "foo(arg1 str, arg2 int)"
        }
    """
)

```json
{
    "error": "HTTP Error",
    "details": "409 Client Error: Conflict for url: http://polaris:8181/api/opendic/v1/objects/function_v2",
    "Catalog Response": null
}
```

In [6]:
catalog.sql(
    """
    SHOW OPEN function_v2
    """
)

,type,name,props,createdTimestamp,lastUpdatedTimestamp,entityVersion
0,function_v2,baz,"{'args': {'arg1': 'int', 'arg2': 'int'}, 'return_type': 'int', 'def': 'def foo(arg1, arg2): return arg1 + arg2', 'signature': 'foo(arg1 str, arg2 int)', 'runtime': '3.12', 'language': 'python', 'comment': 'test fun', 'packages': ['numpy', 'pandas'], 'client_version': 1}",2025-04-23T10:52:06.574956135Z,2025-04-23T10:52:06.574962385Z,1
1,function_v2,foo,"{'args': {'arg1': 'int', 'arg2': 'int'}, 'return_type': 'int', 'def': 'def foo(arg1, arg2): return arg1 + arg2', 'signature': 'foo(arg1 str, arg2 int)', 'runtime': '3.12', 'language': 'python', 'comment': 'test fun', 'packages': ['numpy', 'pandas'], 'client_version': 1}",2025-04-21T17:20:34.843932462Z,2025-04-21T17:20:34.843978462Z,1
2,function_v2,bar,"{'args': {'arg1': 'int', 'arg2': 'int'}, 'return_type': 'int', 'def': 'def foo(arg1, arg2): return arg1 + arg2', 'signature': 'foo(arg1 str, arg2 int)', 'runtime': '3.12', 'language': 'python', 'comment': 'test fun', 'packages': ['numpy', 'pandas'], 'client_version': 1}",2025-04-23T10:27:18.555781335Z,2025-04-23T10:27:18.555784627Z,1


In [1]:
# Show all mappings from <object>. Example: [snowflake,spark]
catalog.sql(
    """
    SHOW OPEN PLATFORMS FOR function_v2
    """
)

NameError: name 'catalog' is not defined

In [15]:
catalog.sql(
    """
    SYNC OPEN function_v2 for snowflake
    """
)

```json
{
    "success": true,
    "executions": [
        {
            "sql": "\"CREATE OR ALTER function_v2 foo(arg1 int, arg2 int)\n            RETURNS int\n            LANGUAGE python\n            RUNTIME = 3.12\n            HANDLER = 'foo'\n            AS \n            $$\n            def foo(arg1, arg2):\n\n        return arg1 + arg2\n            $$\",",
            "status": "failed",
            "error": "001003 (42000): SQL compilation error:\nsyntax error line 1 at position 0 unexpected '\"CREATE OR ALTER function_v2 foo(arg1 int, arg2 int)\n            RETURNS int\n            LANGUAGE python\n            RUNTIME = 3.12\n            HANDLER = 'foo'\n            AS \n            $$\n            def foo(arg1, arg2):\n\n        return arg1 + arg2\n            $$\"'."
        },
        {
            "sql": "\"CREATE OR ALTER function_v2 bar(arg1 int, arg2 int)\n            RETURNS int\n            LANGUAGE python\n            RUNTIME = 3.12\n            HANDLER = 'bar'\n            AS \n            $$\n            def foo(arg1, arg2):\n\n        return arg1 + arg2\n            $$\",",
            "status": "failed",
            "error": "001003 (42000): SQL compilation error:\nsyntax error line 1 at position 0 unexpected '\"CREATE OR ALTER function_v2 bar(arg1 int, arg2 int)\n            RETURNS int\n            LANGUAGE python\n            RUNTIME = 3.12\n            HANDLER = 'bar'\n            AS \n            $$\n            def foo(arg1, arg2):\n\n        return arg1 + arg2\n            $$\"'."
        },
        {
            "sql": "\"CREATE OR ALTER function_v2 baz(arg1 int, arg2 int)\n            RETURNS int\n            LANGUAGE python\n            RUNTIME = 3.12\n            HANDLER = 'baz'\n            AS \n            $$\n            def foo(arg1, arg2):\n\n        return arg1 + arg2\n            $$\",",
            "status": "failed",
            "error": "001003 (42000): SQL compilation error:\nsyntax error line 1 at position 0 unexpected '\"CREATE OR ALTER function_v2 baz(arg1 int, arg2 int)\n            RETURNS int\n            LANGUAGE python\n            RUNTIME = 3.12\n            HANDLER = 'baz'\n            AS \n            $$\n            def foo(arg1, arg2):\n\n        return arg1 + arg2\n            $$\"'."
        }
    ]
}
```

## Drop object and mapping

In [ ]:
catalog.sql(
    """
    DROP OPEN function_v2
    """
)

In [ ]:
catalog.sql(
    """
    DROP OPEN MAPPINGS for snowflake
    """
)